In [6]:
# === Step 2: Audit + Save Summaries in the Group Project Folder ===
import os, pandas as pd, numpy as np
from pathlib import Path
from collections import Counter
from dateutil import parser

# --- Locate the CSV in your current notebook folder ---
csv_candidates = [p for p in Path(".").rglob("Our415_Events_and_Activities_20251008.csv")]
if not csv_candidates:
    raise FileNotFoundError("Couldn't find 'Our415_Events_and_Activities_20251008.csv' in this project folder.")
csv_path = csv_candidates[0]
print(f" Using file: {csv_path}")

# --- Load dataset with headers preserved ---
df = pd.read_csv(csv_path, low_memory=False)
print(f"Loaded {df.shape[0]} rows × {df.shape[1]} columns")

# --- Summaries ---
missing_summary = (
    pd.DataFrame({
        "column": df.columns,
        "missing_count": df.isna().sum().values,
        "missing_pct": (df.isna().sum().values / len(df) * 100).round(2)
    })
    .sort_values(["missing_pct", "column"], ascending=[False, True])
    .reset_index(drop=True)
)

dtype_summary = pd.DataFrame({
    "column": df.columns,
    "pandas_dtype": df.dtypes.astype(str).values,
    "num_unique": [df[c].nunique(dropna=True) for c in df.columns],
    "non_null_count": [df[c].notna().sum() for c in df.columns]
})

def detect_mixed(series, sample_n=400):
    s = series.dropna()
    if s.empty: return {"types": {}, "notes": "all null"}
    if len(s) > sample_n: s = s.sample(sample_n, random_state=42)
    tcount = Counter(type(x).__name__ for x in s)
    notes = ""
    if series.dtype == "object":
        numlike = sum(1 for v in s.astype(str) if v.replace(".","",1).isdigit())
        if 0 < numlike < len(s):
            notes = f"{numlike}/{len(s)} look numeric-like"
    return {"types": dict(tcount), "notes": notes}

mixed_info = [
    {"column": c, **detect_mixed(df[c])} for c in df.columns
]
mixed_types_summary = pd.DataFrame(mixed_info)

id_terms = ("id","uid","guid","event_id","objectid","recordid")
id_candidates = []
for c in df.columns:
    if any(term in c.lower() for term in id_terms):
        nonnull = df[c].notna().sum()
        unique = df[c].nunique(dropna=True)
        id_candidates.append({
            "column": c,
            "non_null_count": nonnull,
            "unique_count": unique,
            "duplicate_count": nonnull - unique,
            "all_non_null_unique": (nonnull == len(df)) and (nonnull == unique)
        })
id_candidates_summary = pd.DataFrame(id_candidates).sort_values(
    ["duplicate_count","non_null_count"], ascending=[True,False]
).reset_index(drop=True)

# --- Create project subfolder for audit artifacts ---
project_dir = Path.cwd()
audit_dir = project_dir / "data_audit"
audit_dir.mkdir(exist_ok=True)
print(f"📁 Saving summaries to: {audit_dir.resolve()}")

# --- Save CSV summaries ---
missing_summary.to_csv(audit_dir / "our415_missingness_summary.csv", index=False)
dtype_summary.to_csv(audit_dir / "our415_dtype_summary.csv", index=False)
mixed_types_summary.to_csv(audit_dir / "our415_mixed_types_summary.csv", index=False)
id_candidates_summary.to_csv(audit_dir / "our415_id_candidates_summary.csv", index=False)

print("\n✅ Audit summaries saved:")
for f in audit_dir.glob("our415_*summary.csv"):
    print(" -", f.name)

# --- Quick preview ---
display(missing_summary.head(10))
display(dtype_summary.head(10))
display(mixed_types_summary.head(10))
display(id_candidates_summary)


 Using file: Our415_Events_and_Activities_20251008.csv
Loaded 2039 rows × 31 columns
📁 Saving summaries to: /Users/nhi/Documents/Documents - Nhi’s MacBook Pro/isys574/Group project/data_audit

✅ Audit summaries saved:
 - our415_missingness_summary.csv
 - our415_dtype_summary.csv
 - our415_id_candidates_summary.csv
 - our415_mixed_types_summary.csv


,column,missing_count,missing_pct
0,admission_price,2039,100.00
1,special_needs_eligibility_tags,2027,99.41
2,race_ethnicity_eligibility_tags,1968,96.52
3,sogi_eligibility_tags,1968,96.52
4,language_eligibility_tags,1833,89.90
5,days_of_week,1088,53.36
6,site_email,1086,53.26
7,subcategory,840,41.20
8,fee,206,10.10
9,age_group_eligibility_tags,172,8.44


,column,pandas_dtype,num_unique,non_null_count
0,id,object,2039,2039
1,org_name,object,14,2026
2,event_name,object,812,2039
3,event_description,object,822,1997
4,event_start_date,object,95,2039
5,event_end_date,object,82,2039
6,days_of_week,object,30,951
7,start_time,object,52,2039
8,end_time,object,54,2039
9,more_info,object,1089,2039


,column,types,notes
0,id,{'str': 400},
1,org_name,{'str': 400},
2,event_name,{'str': 400},
3,event_description,{'str': 400},
4,event_start_date,{'str': 400},
5,event_end_date,{'str': 400},
6,days_of_week,{'str': 400},
7,start_time,{'str': 400},
8,end_time,{'str': 400},
9,more_info,{'str': 400},


,column,non_null_count,unique_count,duplicate_count,all_non_null_unique
0,id,2039,2039,0,True


In [9]:
# === Pre-Cleaning Summary: Category & Subcategory ===
import pandas as pd

# Confirm columns exist
cols = [c for c in df.columns if c.lower() in ["category", "subcategory"]]
if not cols:
    raise ValueError("No 'Category' or 'Subcategory' columns found. Check column names in df.columns.")

# Normalize column casing for internal use (do not modify df)
colmap = {c: c.lower() for c in cols}
temp = df.rename(columns=colmap)

summary_tables = {}
for col in colmap.values():
    s = (
        temp[col]
        .astype(str)
        .fillna("missing")
        .str.strip()
        .value_counts(dropna=False)
        .reset_index()
        .rename(columns={"index": col, col: "count"})
    )
    summary_tables[col] = s
    display(s.head(20))
    print(f"\nTotal unique {col}s:", s.shape[0])

# Save these summaries for reference
audit_dir = Path.cwd() / "data_audit"
(audit_dir / "preclean_category_summary.csv").write_text(
    summary_tables.get("category", pd.DataFrame()).to_csv(index=False)
    if "category" in summary_tables else ""
)
(audit_dir / "preclean_subcategory_summary.csv").write_text(
    summary_tables.get("subcategory", pd.DataFrame()).to_csv(index=False)
    if "subcategory" in summary_tables else ""
)

print(f"\n✅ Saved pre-clean summaries in: {audit_dir}")
for name in ["preclean_category_summary.csv", "preclean_subcategory_summary.csv"]:
    if (audit_dir / name).exists():
        print(" -", name)


,count,count
0,Sports & Recreation,680
1,"Arts, Culture & Identity",540
2,Education,465
3,Childcare,155
4,Family Support,127
5,Health & Wellness,56
6,nan,12
7,"Childcare, Family Support",3
8,"Arts, Culture & Identity, Family Support",1



Total unique categorys: 9


,count,count
0,nan,840
1,Swimming,202
2,Playgroups,121
3,Physical Fitness,97
4,Crafting,88
5,"Arts and Creative Expression, STEM",86
6,Visual Arts,61
7,Support Groups,55
8,"Computer Class, Digital Software, Skills & Tra...",44
9,Writing,41



Total unique subcategorys: 64

✅ Saved pre-clean summaries in: /Users/nhi/Documents/Documents - Nhi’s MacBook Pro/isys574/Group project/data_audit
 - preclean_category_summary.csv
 - preclean_subcategory_summary.csv


In [18]:
# ----- config -----
PRIORITY = [
    "Sports & Recreation",
    "Arts, Culture & Identity",
    "Education",
    "Childcare",
    "Family Support",
    "Health & Wellness",
]

def normalize_and_split(val):
    # handle NaN and the literal "nan"
    if pd.isna(val):
        return []
    s = str(val).strip()
    if not s or s.lower() == "nan":
        return []

    parts = [p.strip() for p in s.split(",") if p.strip()]
    # de-duplicate while preserving order
    parts = list(dict.fromkeys(parts))

    aset = set(parts)
    # --- your explicit rules ---
    if aset == {"Arts, Culture & Identity", "Family Support"}:
        return ["Arts, Culture & Identity"]        # keep Arts, drop Family Support
    if aset == {"Childcare", "Family Support"}:
        return ["Childcare"]                       # keep Childcare, drop Family Support

    return parts

df["categories_list"] = df["category"].apply(normalize_and_split)

In [19]:
category_counts = cat_long["category_clean"].value_counts(dropna=False)
print(category_counts)


category_clean
Sports & Recreation    680
Arts                   541
Culture & Identity     541
Education              465
Childcare              158
Family Support         128
Health & Wellness       56
Name: count, dtype: int64


In [21]:
print(df.columns)


Index(['id', 'org_name', 'event_name', 'event_description', 'event_start_date',
       'event_end_date', 'days_of_week', 'start_time', 'end_time', 'more_info',
       'fee', 'admission_price', 'site_location_name', 'event_photo',
       'category', 'subcategory', 'age_group_eligibility_tags',
       'race_ethnicity_eligibility_tags', 'special_needs_eligibility_tags',
       'sogi_eligibility_tags', 'language_eligibility_tags', 'site_address',
       'site_phone', 'site_email', 'latitude', 'longitude', 'point',
       'analysis_neighborhood', 'supervisor_district', 'data_as_of',
       'data_loaded_at', 'categories_list'],
      dtype='object')


In [22]:
df[["category", "subcategory"]].head(10)


,category,subcategory
0,Sports & Recreation,Swimming
1,Sports & Recreation,Outdoors
2,Education,NaN
3,"Arts, Culture & Identity",Crafting
4,"Arts, Culture & Identity",Culinary Arts; Nutrition Education
5,Sports & Recreation,Outdoors
6,Education,STEM
7,Sports & Recreation,NaN
8,"Arts, Culture & Identity",Crafting
9,Sports & Recreation,NaN


In [23]:
# Save your current cleaned DataFrame as the first audit snapshot
df.to_csv("audit_1_cleaned.csv", index=False)
print("✅ Saved: audit_1_cleaned.csv")


✅ Saved: audit_1_cleaned.csv
